
**Event data QA**

This notebook analyzes Manning's liveBook event data for quality assurance (QA). The assessed data was collected over six months from December 1, 2019 through June 1, 2020. 

---

1. Find popular event types using results from `events_per_account.sql`
2. Check for anomalies in each event with simple time-series plots that use results from `events_per_day.sql`
  - Check if these follow weekly cycles like most real events (weekly behavioural cycles) weekly cycle ot B2C and B2B software use. usage is significantly hier on the weekdays and much slowero nthe weekends. calc % use diff between weekend and weekday to confirm
 

- Basic customer metrics
- Create customer dataset



In [1]:
!pip install plotly --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import auth
from google.auth import default
import gspread
import pandas as pd
import plotly.express as px

In [3]:
auth.authenticate_user()

creds, _ = default()

# Save events_per_account.gsheet as a Pandas DataFrame
gc = gspread.authorize(creds)
worksheet = gc.open('events_per_account').sheet1
rows = worksheet.get_all_values() 
account_metrics = pd.DataFrame.from_records(rows)


### Find popular events with `events_per_account`

---
 
- What event types are most common?
- What event types are least common?
- What event types do customers average at least 1 event per month?


---

**Prepare `account_metrics` DataFrame**

This DataFrame contains the number of events per account and events per account per month for each type of liveBook event.

In [4]:
# Set header
account_metrics.columns = rows[0]
account_metrics = account_metrics[1:]

In [5]:
# Convert numerical columns
num_cols = ['event_count', 'account_count', 'events_per_account', 'month_count', 'events_per_account_per_month']
account_metrics[num_cols] = account_metrics[num_cols].apply(pd.to_numeric, errors='coerce')
# Verify 
account_metrics.dtypes

event_type                       object
event_count                       int64
account_count                     int64
events_per_account              float64
month_count                       int64
events_per_account_per_month    float64
dtype: object

In [6]:
# Compute event frequency 
event_total = account_metrics.event_count.sum() # 3156212
account_metrics['event_pct'] = account_metrics.apply(lambda x: x.event_count/event_total*100, axis=1)

Over 3 million (3156212) liveBook events were logged by 88375 active accounts over the 6-month measurement period.


**Find popular events**

In [7]:
# Sort by `events_per_account_per_month`
account_metrics.sort_values(by='events_per_account_per_month', ascending=True, inplace=True)

# Plot
fig = px.bar(account_metrics, y='event_type', x='events_per_account_per_month', title='Popular events', height=1000)
fig.update_xaxes(title_text=None, showgrid=False, nticks=10, ticks="outside", tickcolor='#003f5c', ticklen=3)
fig.update_yaxes(title_text=None, ticks="outside", tickcolor='white', ticklen=10)
fig.update_layout( 
    font_family='Open Sans', 
    xaxis={'side': 'top'},
    showlegend=False,
    hoverlabel=dict(
        bgcolor="white",
        font_size=10))
fig.add_vrect(x0=0, x1=0.1, fillcolor="white", opacity=0.35, layer="above", line_width=0)
fig.update_traces(hovertemplate="%{x} %{y} events per account per month")
fig.show()

There are 10 types of events on the liveBook platform that averaged at least 0.1 events per account per month. 

In descending order from most frequent to least frequent, they are:

1. `ReadingOwnedBook`
2. `FirstLivebookAccess`
3. `FirstManningAccess`
4. `EBookDownloaded`
5. `ReadingFreePreview`
6. `FreeContentCheckout`
7. `HighlightCreated`
8. `ReadingOpenChapter`
9. `ProductTocLivebookLinkOpened`
10. `LivebookLogin`

Note that events related to reading are amongst the most popular on the liveBook platform, which makes sense. Reading-related events include: `ReadingOwnedBook`, `ReadingFreePreview`, `EBookDownloaded`, `ReadingOpenChapter`.

The least popular event types are 

In [9]:
popular_events = account_metrics['event_type'].loc[account_metrics['events_per_account_per_month'] > 0.1].tolist()
len(popular_events)
popular_events

['LivebookLogin',
 'ProductTocLivebookLinkOpened',
 'ReadingOpenChapter',
 'HighlightCreated',
 'FreeContentCheckout',
 'ReadingFreePreview',
 'EBookDownloaded',
 'FirstManningAccess',
 'FirstLivebookAccess',
 'ReadingOwnedBook']


### `events_per_day`

---

- Do events happen equally every day, or are there patterns?
- Are there any gaps in the record of any events?
- Are there any events that only occur in part of the history?
- Are there any extreme outliers or anomalies in the number of events?




In [10]:
import itertools
from plotly.subplots import make_subplots
import plotly.graph_objects as go

r = [i for i in range(1,6)]
c = [i for i in range(1,3)]
pos = list(itertools.product(r, c))

fig = make_subplots(rows=r[-1], cols=c[-1], subplot_titles=popular_events)

for event, p in zip(popular_events, pos):
  filepath = f"/content/drive/MyDrive/Churn prediction/events_per_day_{event}.csv"
  event_df = pd.read_csv(filepath, index_col='measurement_date')
  fig.append_trace(
      go.Scatter(x=event_df.index, y=event_df['event_count'],  line=dict(width=0.5), marker_color='darkslategray'),
      row=p[0], col=p[1])
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(height=1000, width=1000, title_text="Anomaly detection", showlegend=False)
fig.show()

Big-picture observations 

*   Some events like FirstLivebookAccess and FirstManningAccess
*   List item



In [33]:
all_events = []
for event in popular_events:
    filepath = f"/content/drive/MyDrive/Churn prediction/events_per_day_{event}.csv"
    event_df = pd.read_csv(filepath, index_col='measurement_date')
    all_events.append(event_df)

all_events = pd.concat(all_events, axis=1, join="outer")
all_events.columns = popular_events

# Verify
all_events.head(4)

,LivebookLogin,ProductTocLivebookLinkOpened,ReadingOpenChapter,HighlightCreated,FreeContentCheckout,ReadingFreePreview,EBookDownloaded,FirstManningAccess,FirstLivebookAccess,ReadingOwnedBook
measurement_date,,,,,,,,,,
2019-12-01,1,387,503,577,1,502,1,1,1,2549
2019-12-02,1,687,690,886,1,729,1,1,1,3556
2019-12-03,1,411,610,951,1,596,1,1,1,3581
2019-12-04,190,483,597,659,1,671,2,1,1,3544


In [67]:
buttons_list = []

for event in popular_events:
  button = {}
  button["args"] = [{
      'y': [all_events[event]],
      'visible': True}, 
       {'title':event}, [0]]
  button["label"] = event
  button["method"] = "update"
  buttons_list.append(button)

In [85]:
fig = go.Figure()
fig.add_traces(go.Scatter(x=all_events.index, y=all_events.LivebookLogin, visible=True))

fig.update_yaxes(
    showgrid=False)

fig.update_xaxes(
    showgrid=False,
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(step="all")])))

fig.update_layout(
    title="LivebookLogin",
    updatemenus=[
        dict(
            buttons=buttons_list,
            direction="down",
            showactive=True,
            x=1,
            xanchor="right",
            y=1.16,
            yanchor="top")])
fig.show()